# Desafio Nasa 1

O objetivo deste notebook é responder as perguntas descritas a seguir:

1. Número de hosts únicos ?
2. O total de erros 404 ?
3. Os 5 URLs que mais causaram erro 404 ?
4. Quantidade de erros 404 por dia ?
5. O total de bytes retornados ?


Para obtenção das respostas, serão usados o seguintes datasets

 - (https://github.com/paaarx/semantix/blob/master/NASA_access_log_Aug95.gz)
 - (https://github.com/paaarx/semantix/blob/master/NASA_access_log_Jul95.gz)

In [ ]:
%sh

pwd

/databricks/driver


In [ ]:
%sh ls -l

total 816
drwxr-xr-x 2 root root   4096 Nov  8 06:22 conf
drwxr-xr-x 3 root root   4096 Nov  9 11:35 eventlogs
drwxr-xr-x 2 root root   4096 Nov  9 14:15 ganglia
drwxr-xr-x 2 root root   4096 Nov  9 14:01 logs
drwxr-xr-x 5 root root   4096 Nov  9 11:41 metastore_db
-r-xr-xr-x 1 root root 813069 Nov  8 06:22 preload_class.lst


## Obtendo conjunto de dados - Extract

In [ ]:
%sh wget https://github.com/paaarx/semantix/raw/master/NASA_access_log_Aug95.gz

--2021-11-09 14:42:39--  https://github.com/paaarx/semantix/raw/master/NASA_access_log_Aug95.gz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/paaarx/semantix/master/NASA_access_log_Aug95.gz [following]
--2021-11-09 14:42:39--  https://raw.githubusercontent.com/paaarx/semantix/master/NASA_access_log_Aug95.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16633316 (16M) [application/octet-stream]
Saving to: ‘NASA_access_log_Aug95.gz’

     0K .......... .......... .......... .......... ..........  0% 6.66M 2s
    50K .......... .......... .......... .......... ..........  0% 4.72M 3s
   100

In [ ]:
%sh wget https://github.com/paaarx/semantix/raw/master/NASA_access_log_Jul95.gz

--2021-11-09 14:42:46--  https://github.com/paaarx/semantix/raw/master/NASA_access_log_Jul95.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/paaarx/semantix/master/NASA_access_log_Jul95.gz [following]
--2021-11-09 14:42:46--  https://raw.githubusercontent.com/paaarx/semantix/master/NASA_access_log_Jul95.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20676672 (20M) [application/octet-stream]
Saving to: ‘NASA_access_log_Jul95.gz’

     0K .......... .......... .......... .......... ..........  0% 5.16M 4s
    50K .......... .......... .......... .......... ..........  0% 2.81M 5s
   100

In [ ]:
%sh ls -lh

total 37M
-rw-r--r-- 1 root root  16M Nov  9 14:42 NASA_access_log_Aug95.gz
-rw-r--r-- 1 root root  20M Nov  9 14:42 NASA_access_log_Jul95.gz
drwxr-xr-x 2 root root 4.0K Nov  8 06:22 conf
drwxr-xr-x 3 root root 4.0K Nov  9 11:35 eventlogs
drwxr-xr-x 2 root root 4.0K Nov  9 14:30 ganglia
drwxr-xr-x 2 root root 4.0K Nov  9 14:01 logs
drwxr-xr-x 5 root root 4.0K Nov  9 11:41 metastore_db
-r-xr-xr-x 1 root root 795K Nov  8 06:22 preload_class.lst


In [ ]:
! gzip -lv ./NASA_access_log_Aug95.gz

method  crc     date  time           compressed        uncompressed  ratio uncompressed_name
defla 89529258 Nov  9 14:42            16633316           167813770  90.1% ./NASA_access_log_Aug95


In [ ]:
! gzip -lv ./NASA_access_log_Jul95.gz

method  crc     date  time           compressed        uncompressed  ratio uncompressed_name
defla 6656c0d8 Nov  9 14:42            20676672           205242368  89.9% ./NASA_access_log_Jul95


In [ ]:
%sh gzip -dk ./NASA_access_log_Aug95.gz

In [ ]:
%sh gzip -dk ./NASA_access_log_Jul95.gz

In [ ]:
%sh ls -lh

total 393M
-rw-r--r-- 1 root root 161M Nov  9 14:42 NASA_access_log_Aug95
-rw-r--r-- 1 root root  16M Nov  9 14:42 NASA_access_log_Aug95.gz
-rw-r--r-- 1 root root 196M Nov  9 14:42 NASA_access_log_Jul95
-rw-r--r-- 1 root root  20M Nov  9 14:42 NASA_access_log_Jul95.gz
drwxr-xr-x 2 root root 4.0K Nov  8 06:22 conf
drwxr-xr-x 3 root root 4.0K Nov  9 11:35 eventlogs
drwxr-xr-x 2 root root 4.0K Nov  9 15:30 ganglia
drwxr-xr-x 2 root root 4.0K Nov  9 15:01 logs
drwxr-xr-x 5 root root 4.0K Nov  9 11:41 metastore_db
-r-xr-xr-x 1 root root 795K Nov  8 06:22 preload_class.lst


In [ ]:
! head -n 2 NASA_access_log_Aug95

in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839
uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0


In [ ]:
! head -n 2 NASA_access_log_Jul95

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985


In [ ]:
%sh mkdir Nasa_data

In [ ]:
%sh

mv ./NASA_access_log_Jul95 ./Nasa_data/NASA_access_log_0Jul95
mv ./NASA_access_log_Aug95 ./Nasa_data/NASA_access_log_1Aug95

**Observação**

Uma possível melhor prática talvez seja criar um programa em Python ou *Shell Script* para checar as informações dos arquivos de log e selecionar apenas as informações de interesse.
Contudo, serão usados os recursos do Pyspark para aplicar os tratamentos necessários aos dados.

In [ ]:
df_raw1 = (
  spark.read.option("sep", " ")
  .option("header", False)
  .option("inferSchema", True)
  .csv("file:/databricks/driver/Nasa_data")
)

In [ ]:
display(df_raw1)

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7
199.72.81.55,-,-,[01/Jul/1995:00:00:01,-0400],GET /history/apollo/ HTTP/1.0,200,6245
unicomp6.unicomp.net,-,-,[01/Jul/1995:00:00:06,-0400],GET /shuttle/countdown/ HTTP/1.0,200,3985
199.120.110.21,-,-,[01/Jul/1995:00:00:09,-0400],GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0,200,4085
burger.letters.com,-,-,[01/Jul/1995:00:00:11,-0400],GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0
199.120.110.21,-,-,[01/Jul/1995:00:00:11,-0400],GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0,200,4179
burger.letters.com,-,-,[01/Jul/1995:00:00:12,-0400],GET /images/NASA-logosmall.gif HTTP/1.0,304,0
burger.letters.com,-,-,[01/Jul/1995:00:00:12,-0400],GET /shuttle/countdown/video/livevideo.gif HTTP/1.0,200,0
205.212.115.106,-,-,[01/Jul/1995:00:00:12,-0400],GET /shuttle/countdown/countdown.html HTTP/1.0,200,3985
d104.aa.net,-,-,[01/Jul/1995:00:00:13,-0400],GET /shuttle/countdown/ HTTP/1.0,200,3985
129.94.144.152,-,-,[01/Jul/1995:00:00:13,-0400],GET / HTTP/1.0,200,7074


In [ ]:
df_raw1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



## Tratamento dos dados - Transform

Ao se observar os dados, descobre-se
- A coluna "_c0" contém a origem da requisição
- A coluna "_c1" e "_c2" não contém informação relevante
- A coluna "_c3" e "_c4" contém a data da requisição
- A coluna "_c5" contém o comando http, caminho da requisição e o protocólo utilizado
- A coluna "_c6" contém o número de status da requisição
- A coluna "_c7" contém o total de bytes retornados.

Algumas verificações nos dados

In [ ]:
df_raw1.filter("_c1 != '-' or _c2 != '-'").count()

Out[57]: 0

In [ ]:
df_raw1.filter( df_raw1._c6.rlike('[^0-9]') ).count()

Out[58]: 32

In [ ]:
df_raw1.filter( df_raw1._c7.rlike('[^0-9]') ).count()

Out[59]: 33875

In [ ]:
display(df_raw1.filter( df_raw1._c6.rlike('[^0-9]') ))

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7
frank.mtsu.edu,-,-,[03/Jul/1995:02:41:15,-0400],GET /images/,"HTTP/1.0""",404
frank.mtsu.edu,-,-,[03/Jul/1995:02:47:17,-0400],GET /images/,"HTTP/1.0""",404
frank.mtsu.edu,-,-,[03/Jul/1995:02:48:51,-0400],GET /images/,"HTTP/1.0""",404
frank.mtsu.edu,-,-,[03/Jul/1995:02:50:27,-0400],GET /images,"HTTP/1.0""",404
141.102.80.253,-,-,[07/Jul/1995:11:57:49,-0400],GET /history/apollo/apollo.html,"HTTP/1.0""",404
141.102.80.130,-,-,[12/Jul/1995:13:42:32,-0400],GET /history/apollo/apollo.html,"HTTP/1.0""",404
141.102.80.130,-,-,[13/Jul/1995:19:25:35,-0400],GET /history/apollo/apollo.html,"HTTP/1.0""",404
ris1_u27.rsoc.rockwell.com,-,-,[14/Jul/1995:10:37:53,-0400],GET /history/apollo/apollo.html,"HTTP/1.0""",404
piweba3y.prodigy.com,-,-,[14/Jul/1995:23:46:07,-0400],"""GET /history/appollo/appollo-13/html-""more%20info""","HTTP/1.0""",404
arc.dental.upenn.edu,-,-,[18/Jul/1995:11:53:44,-0400],"""GET /elv/elvpage.htm/""MISSION""","HTTP/1.0""",403


Após a verificação, constatou-se que houve perda de informações após a importação dos arquivos de log pelo Spark.